In [ ]:
import cv2
import numpy as np

# Detail metrics

def edge_density(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 100, 200)
    density = np.sum(edges > 0) / edges.size
    return density

def laplacian_variance(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return cv2.Laplacian(img, cv2.CV_64F).var()

from skimage import io, color
from skimage.measure import shannon_entropy

def image_entropy(image_path):
    img = io.imread(image_path)

    # Handle alpha channel if present
    if img.ndim == 3 and img.shape[2] == 4:  # RGBA
        img = img[:, :, :3]  # Drop alpha

    # Convert grayscale to 3-channel for consistency
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)

    # Normalize to [0, 1] and convert to grayscale
    img = img.astype("float32") / 255.0
    gray = color.rgb2gray(img)

    return shannon_entropy(gray)

In [ ]:
# Color metrics

# Hasler and Süsstrunk’s Colorfulness Metric
def colorfulness(image_path):
    img = cv2.imread(image_path)
    (B, G, R) = cv2.split(img.astype("float"))
    rg = np.absolute(R - G)
    yb = np.absolute(0.5 * (R + G) - B)

    std_rg, std_yb = np.std(rg), np.std(yb)
    mean_rg, mean_yb = np.mean(rg), np.mean(yb)

    return np.sqrt(std_rg**2 + std_yb**2) + 0.3 * np.sqrt(mean_rg**2 + mean_yb**2)

from scipy.stats import entropy

# How evenly colors are distributed. High entropy = uniform spread
# (no dominant color), low entropy = color is concentrated in few bins.
def _histogram_spread(channel, bins=32):
    hist, _ = np.histogram(channel, bins=bins, range=(0, 1), density=False)
    non_zero_bins = np.count_nonzero(hist)
    return non_zero_bins / bins  # value in [0, 1]

def _histogram_entropy(channel, bins=32):
    hist, _ = np.histogram(channel, bins=bins, range=(0, 1), density=True)
    return entropy(hist + 1e-10)  # add epsilon to avoid log(0)

def color_histogram_diversity(image_path, color_space='hsv', bins=32):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

    if color_space == 'hsv':
        converted = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2HSV) / 255.0
    elif color_space == 'lab':
        converted = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2Lab) / 255.0
    else:
        raise ValueError("Choose 'hsv' or 'lab'.")

    diversity_metrics = []
    for i in range(3):
        channel = converted[:, :, i].flatten()
        spread = _histogram_spread(channel, bins)
        ent = _histogram_entropy(channel, bins)
        diversity_metrics.append((spread, ent))

    # Combine (you can weight them differently if needed)
    avg_spread = np.mean([m[0] for m in diversity_metrics])
    avg_entropy = np.mean([m[1] for m in diversity_metrics])
    return {
        "avg_spread": avg_spread,
        "avg_entropy": avg_entropy
    }
    
img = rf'dataset\mery\danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(color_histogram_diversity(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(color_histogram_diversity(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(color_histogram_diversity(img))

In [ ]:
#Temperature analysis

# Computes the mean and stddev of color temps in an image, from -255 (blue) to 255 (red)
def temperature_analysis(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert BGR to RGB (since OpenCV loads images in BGR format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extract Red and Blue channels
    red_channel = image[:, :, 0].astype(np.int16)
    blue_channel = image[:, :, 2].astype(np.int16)

    # Compute difference (R - B)
    diff = red_channel - blue_channel

    # Compute statistics
    temp_mean = np.mean(diff)
    temp_stddev = np.std(diff)

    return {
        "temp_mean": temp_mean,
        "temp_stddev": temp_stddev
    }

img = rf'dataset\mery\danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(temperature_analysis(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(temperature_analysis(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(temperature_analysis(img))

{'temp_mean': 44.7559594095941, 'temp_stddev': 43.500473568086576}
{'temp_mean': -21.0788, 'temp_stddev': 38.8967889492179}
{'temp_mean': 25.513930817610063, 'temp_stddev': 23.606374944234716}


In [25]:
# Values and Contrast
import numpy as np
from skimage import io, color

def grayscale_contrast_analysis(image_path):
    # Load image
    img = io.imread(image_path)

    # Handle alpha channel or grayscale input
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]
    elif img.ndim == 2:
        img = np.stack([img]*3, axis=-1)

    # Convert to grayscale [0, 1]
    img = img.astype("float32") / 255.0
    gray = color.rgb2gray(img)

    # Compute mean and std deviation
    mean_val = np.mean(gray)
    std_val = np.std(gray)

    return {
        "mean_gray": mean_val,
        "std_gray": std_val  # Higher = more contrast
    }

img = rf'dataset\mery\danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(grayscale_contrast_analysis(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(grayscale_contrast_analysis(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(grayscale_contrast_analysis(img))

{'mean_gray': 0.76921487, 'std_gray': 0.22880471}
{'mean_gray': 0.6635087, 'std_gray': 0.3146969}
{'mean_gray': 0.75491685, 'std_gray': 0.25347784}
